# Bug fixes to do
- To fix the rainfall and supply calculation code. 

I am assuming that the rainfall through the gauges will be for the entire past day

- def updateDemandOfWater() --> this function has to be coded

In [74]:
import numpy as np
import pandas as pd
import pulp


In [75]:
n=10
m=5
seti= range(n)
setj = range(1,m)
fcosts = {}
fcosts['lambda'] = 10000 # per km for pipe
fcosts['cost_of_tank'] = 1000 # per liters increase
fcosts['stability_inc_cost'] = 10000
beta = fcosts['stability_inc_cost']


In [76]:
# house class
class house:
    def __init__(self):
        self.lat = 0
        self.lng = 0
        self.height = 0
        self.district = ''
        self.city = ''
        self.pincode = 0
        self.demand = [] # may be average monthly demand
        self.supply = 0
        self.rooftopArea = 0.0 # meter square
        self.catchmentArea = 0.0 # meter square 
        self.pipesToTank = {} # j: distance and cost
        self.gague = None # gauge object that is affected by the rainfall
        
    def helperfunctions(self):
        pass
        
    def calculateSupply(self):
        if self.gague == None:
            print("the gauge is not yet pointed")
            return
        else:
            rainfall = self.gauge.rainfallInMM
            self.supply = rainfall*(self.catchmentArea + self.rooftopArea) # we do have to check this
            
class hub:
    def __init__(self):
        pass
# tank class for storage
class tank:
    def __init__(self):
        self.lat = 0
        self.lng = 0
        self.height = 0
        self.district = ''
        self.city = ''
        self.pincode = 0
        self.capacity = []
        self.stability = 1
        self.pipesToTank = {} # j: distance and cost
        
    def helperfunctions(self):
        pass

# gauge to calc rainfall   
class gauge:
    def __init__(self):
        self.lat = 0
        self.lng = 0
        self.height = 0
        self.district = 0
        self.otherReadings = None # there may be other things assocated to a a gague
        self.rainfallInMM = 0.0
    def updateRainfall(rainfall):
        self.rainfallInMM = rainfall
        

In [77]:
gauges = [gauge() for _ in range(3)] # say there are 2 gauges in the area
houses = [house() for _ in range(n)]
tanks = [tank() for _ in range(m)]



In [78]:
# kmeans clustering of houses
def clusterHousesAndCalculateWaterSupply(houses, gauges):
    for h_idx in range(len(houses)):
        # x, y,z for houses
        x_h = houses[h_idx].lat
        y_h = houses[h_idx].lng
        z_h = houses[h_idx].height
        
        minDistance = np.inf
        minDistanceGauge = 0
        for g_idx in range(len(gauges)):
            # x,y,z for gauges
            x_g = gauses[g_idx].lat
            y_g = gauges[g_idx].lng
            z_g = gauges[g_idx].height
            
            # distance calculated from each gauge
            distance= (x_h-x_g)**2 + (y_h-y_g)**2 + (z_h-z_g)**2
            
            # update distance
            if(distance < minDistance):
                minDistance = distance
                minDistanceGauge = g_idx
        
        # rainfall evaluation to calculate supply of water from the rain the previous day
        houses[h_idx].gauge = gauges[minDistanceGauge]
        houses[h_idx].calculateSupply()
        print('House objects now contain the supply of water')
        
def updateDemandOfWater_HOUSES(houses, waterDemandedArrayFromGoldmanSachs):
    # This depends on the data 
    pass

def updateLatLngHeight_HOUSES(houses, dataFromGodmanSachs):
    # This depends on the data 
    pass

def updateDistrictCityPinCode_HOUSES(houses, dataFromGoldmanSachs):
    # This depends on the data 
    pass

def updateDemandOfWater_TANKS(tanks, waterDemandedArrayFromGoldmanSachs):
    # This depends on the data 
    pass

def updateLatLngHeightHouses_TANKS(tanks, dataFromGodmanSachs):
    # This depends on the data 
    pass

def updateDistrictCityPinCode_TANKS(tanks, dataFromGoldmanSachs):
    # This depends on the data 
    pass

def updateStability_TANKS(tanks, datafromGoldmanSachs):
    # this depends on the data
    pass

In [79]:
10 in range(10)

False

In [80]:
# known variables

# supply i denotes supply of water on any given day from houses
Si = {
    i:houses[i].supply for i in seti
}

# Dij is distance from ith house to jth tank
Dij = {
    (i,j): (houses[i].lat - tanks[j].lat)**2 + (houses[i].lng - tanks[j].lng)**2 +(houses[i].height - tanks[j].height)**2 
    for i in seti for j in setj
    
}

# FCj is fixed cost of jth tank
FCj = {
    j: tanks[j].stability * beta + fcosts["cost_of_tank"] for j in setj
}

In [81]:
import pulp as plp

In [82]:
# decision variables

# pipe from i to j
Pij = {
    (i,j): plp.LpVariable(
        cat = plp.LpBinary,
        name = "P_{0}_{1}".format(i,j)
    ) for i in seti for j in setj
}

# # if there is a tank or not
yj = {
    j:plp.LpVariable(
        cat = plp.LpBinary,
        name = "Y_{0}".format(j)
    ) for j in setj
}

# # storange capacity of tank j
xj = {
    j: plp.LpVariable(
        cat = plp.LpContinuous,
        lowBound = 0,
        upBound = None, # no upper bound
        name = "x_{0}".format(j)
    ) for j in setj
}

In [98]:
# setting up the constraints

list_constraint_2 = {
    j : plp.LpConstraint(
        e= plp.lpSum(Si[i]*Pij[i,j] for i in seti),
        sense = plp.LpConstraintLE,
        rhs = xj[j]
    ) for j in setj
}

In [101]:
optimization_model = plp.LpProblem(name = "MLIPP_GS_Problem")

In [105]:
objective = plp.lpSum(
    plp.lpSum((yj[j] * (FCj[j] + fcosts["cost_of_tank"]*xj[j])) for j in setj) + plp.lpSum(yj[j] * Dij[i][j]*fcosts["lambda"] for i in seti for j in setj))
                

TypeError: Non-constant expressions cannot be multiplied

In [100]:
optimization_model.sense = plp.LpMinimize
optimization_model.setObjective(objective)
optimization_model.solve()

NameError: name 'objective' is not defined